## Trackpy tracking 

Tracking of cell movement in chemotaxis experiment. 
Must import: 

* the bckg subtracted video 
* the pipette properties dictionary 

Gives as output: 

* **df_up**, **df_low**: a dataframe for each pipette containing for each tracked particle the information on: 
  x-y coordinates, x-y coordinates in the reference system centered at the pipette entrance (x_new, y_new), x-velocity, y-velocity, module of velocity, angle of cell velocity vector with resepect to the radial direction centered in the pipette center 
  
Must check that the tracking parameters are good before running this script!

*Written by Medea Zanoli 30/12/2021*

In [1]:
from PIL import Image, ImageEnhance, ImageSequence, ImageOps,  ImageChops, ImageFilter,  ImageDraw
from tkinter import Tk
from tkinter.filedialog import askdirectory, askopenfilename
import os
import glob
import matplotlib.pyplot as plt
%matplotlib inline 
import numpy as np
from progressbar import ProgressBar
import progressbar
import pandas as pd
import cv2 
import seaborn as sns
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
from matplotlib import path
import pickle
import pims
import trackpy as tp
from scipy import interpolate


# define the function to calculate the angle
def angle_function(unit_r, unit_v): 
    
    # check for cross product 
    cross = np.cross(unit_r, unit_v)
    
    if(cross >= 0 ): 
        angle =  np.rad2deg(np.arccos(np.dot(unit_r, unit_v))) 
    else: 
        angle =  - np.rad2deg(np.arccos(np.dot(unit_r, unit_v)))
    return angle

### Import the black and white background substracted video 

In [8]:
# FIRST IMPORT THE VIDEO TO PROCESS

# background substracted video
video_file = askopenfilename()
video = pims.as_gray(pims.Video(video_file))
bw_cap= cv2.VideoCapture(video_file)

In [10]:
frame_number = len(video) 
bar = ProgressBar(maxval=frame_number).start()

i=0
bw_frame_list = []

while(bw_cap.isOpened()):
    ret, frame = bw_cap.read()
    if ret == False:
        break
        
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    bw_frame_list.append(gray_frame)
    i+=1
    bar.update(i)
    
print('total number of frames: %i' %frame_number)

total number of frames: 110


### Import the pipette properties

In [11]:
# select the pipette properties file to use 
pipette_properties_file = askopenfilename() 
a_file = open(pipette_properties_file, "rb")
pipette_properties_dictionary = pickle.load(a_file)

### Select the folder where to save the outputs 

In [7]:
path_for_saving = askdirectory()

### Set the tracking parameters

In [14]:
# parameters for the tracking 
pixel_number = 11 #11#11 # diameter: features extent. Must be an be odd integer. When in doubt, round up.
                    #If we use a pixel number too high then it groups together cells that are too close to each other. 
threshold = 30 #30 # Clip bandpass result below this value (correspond to the parameter "signal")
separation = 0.5#1# 0.1 Minimum separation between maxima
minmass = 1000 #  "mass" means total integrated brightness of the blob

search_range = 5 #the farthest a particle can travel between frames (search_range): maximum distance
memory = 3

In [15]:
# DO THE TRACKING 
# check features in all frames 

print('total number of frames %i' %frame_number)
print('checking features in all frame...')
ff= tp.batch(video, pixel_number, threshold = threshold, minmass=minmass, processes=1, separation=separation)
    
print('linking the trajectories between frames...')
# track trajectories 
t = tp.link(ff, search_range= search_range, memory=memory) 

# filter out trajectories shorter than threshold 
t = tp.filtering.filter_stubs(t, threshold=4)

Frame 109: 214 trajectories present.


In [17]:
# CALCULATE NEW COORDINATES, VELOCITIES AND ANGLES

# calculate x_new, y_new, velocity components and angles 

# set frame as index
df_i=t.set_index('frame')

# add a time column in seconds
fps = 30 # frames per seconds of original video
df_i.loc[:, 'time']= df_i.index*(1/fps) # time in seconds

# define 2 daframes: one for the upper pipette, and one for the lower 
df_up = df_i.copy()
df_low = df_i.copy()

# export from the dictionary the pipette properties 
upper_contour = pipette_properties_dictionary['upper_contour']
lower_contour = pipette_properties_dictionary['lower_contour']

theta_up = pipette_properties_dictionary['theta_up']
theta_low = pipette_properties_dictionary['theta_low']

x_center_pipette_up = pipette_properties_dictionary['x_center_upper_pipette']
y_center_pipette_up = pipette_properties_dictionary['y_center_upper_pipette']

x_center_pipette_low = pipette_properties_dictionary['x_center_lower_pipette']
y_center_pipette_low = pipette_properties_dictionary['y_center_lower_pipette']

# new coordinates in the reference system centered in the upper pipette
x_new_up = (df_i.x - x_center_pipette_up)*np.cos(theta_up) - (df_i.y - y_center_pipette_up)*np.sin(theta_up)

y_new_up = (df_i.x - x_center_pipette_up)*np.sin(theta_up) +(df_i.y - y_center_pipette_up)*np.cos(theta_up)
y_new_up = - y_new_up ## ->> so that the new coordinate sistems points UP 

# new coordinates in the reference system in the lower pipette 
x_new_low = (df_i.x - x_center_pipette_low)*np.cos(theta_low) - (df_i.y - y_center_pipette_low)*np.sin(theta_low)

y_new_low = (df_i.x - x_center_pipette_low)*np.sin(theta_low) +(df_i.y - y_center_pipette_low)*np.cos(theta_low)
y_new_low = - y_new_low ## ->> so that the new coordinate sistems points UP 

# add empty columns where we will put vx, vy and angle
empty_array = np.empty(np.shape(df_i)[0])
empty_array[:] = np.nan

df_up.loc[:, 'x_new'] = x_new_up
df_up.loc[:, 'y_new'] = y_new_up
df_up.loc[:, 'r'] = np.sqrt(x_new_up**2 + y_new_up**2)
df_up.loc[:, 'vx'] = empty_array
df_up.loc[:, 'vy'] = empty_array
df_up.loc[:, 'v'] = empty_array
df_up.loc[:, 'angle'] = empty_array

df_low.loc[:, 'x_new'] = x_new_low
df_low.loc[:, 'y_new'] = y_new_low
df_low.loc[:, 'r'] = np.sqrt(x_new_low**2 + y_new_low**2)
df_low.loc[:, 'vx'] = empty_array
df_low.loc[:, 'vy'] = empty_array
df_low.loc[:, 'v'] = empty_array
df_low.loc[:, 'angle'] = empty_array

# perform a spline interpolation for getting x and y derivatives --> vx and vy
smoothing_factor = 0

bar = ProgressBar(max_value= len(np.unique(df_i.particle) + 1 )

print('total number of particle = % i' %np.max(np.unique(df_i.particle)))

print('processing df_up')

# compute derivatives for upper pipette

for particle in bar(np.unique(df_i.particle)):
    # select the part of dataframe corresponding to the selected particle 
    particle_df = df_up[df_up['particle'] == particle]

    # spline interpolation
    # tck --> A tuple (t,c,k) containing the vector of knots, the B-spline coefficients, and the degree of the spline
    # by default spline order = 3
    tck_y = interpolate.splrep(particle_df.time, particle_df.y_new, s= smoothing_factor) 
    tck_x = interpolate.splrep(particle_df.time, particle_df.x_new, s= smoothing_factor)
      
     # x and y first derivative evaluated at all positions
    xders = interpolate.splev(particle_df.time, tck_x, der = 1) # x derivative 
    yders = interpolate.splev(particle_df.time, tck_y, der = 1) # y derivative
    
    # vector contaning the vx and vy of the particle in all frames 
    velocity_direction_vector = np.transpose([xders, yders])  
    # and normalize it 
    velocity_direction_vector = [row_element/np.linalg.norm(row_element) for row_element in velocity_direction_vector ]
         
    # unit vector pointing in the radial direction to the center of the pipette
    radial_direction_vector =   np.transpose([-particle_df.x_new, -particle_df.y_new])  
    radial_direction_vector = [row_element/np.linalg.norm(row_element) for row_element in radial_direction_vector ]
      
    # the angle is calculated as the arcos of the dot product between two arrays: an array pointing in the radial direction and the 
    # one of the instantaneus velocity of the bicho
    angles = [ angle_function(unit_r, unit_v) for unit_r, unit_v  in zip(radial_direction_vector, velocity_direction_vector) ]    
        
    df_up.loc[df_up['particle'] == particle, 'angle'] =angles
    df_up.loc[df_up['particle'] == particle, 'vx'] =xders
    df_up.loc[df_up['particle'] == particle, 'vy'] =yders
    df_up.loc[df_up['particle'] == particle, 'v'] = np.sqrt(xders**2 + yders**2)
        
    
# ... and for the lower 
bar = ProgressBar(max_value=progressbar.UnknownLength)

print('processing df_low')

for particle in bar(np.unique(df_i.particle)):
    #print('processing particle %i' %particle)
    

    # select the part of dataframe corresponding to the selected particle 
    particle_df = df_low[df_low['particle'] == particle]
     
    # spline interpolation
    tck_y = interpolate.splrep(particle_df.time, particle_df.y_new, s= smoothing_factor) 
    tck_x = interpolate.splrep(particle_df.time, particle_df.x_new, s= smoothing_factor)
      
     # x and y first derivative evaluated at all positions
    xders = interpolate.splev(particle_df.time, tck_x, der = 1) # x derivative 
    yders = interpolate.splev(particle_df.time, tck_y, der = 1) # y derivative
    
    # vector contaning the vx and vy of the particle in all frames 
    velocity_direction_vector = np.transpose([xders, yders])  
    # and normalize it 
    velocity_direction_vector = [row_element/np.linalg.norm(row_element) for row_element in velocity_direction_vector ]
      
    # unit vector pointing in the radial direction to the center of the pipette
    radial_direction_vector =   np.transpose([-particle_df.x_new, -particle_df.y_new])  
    radial_direction_vector = [row_element/np.linalg.norm(row_element) for row_element in radial_direction_vector ]
      
    # the angle is calculated as the arcos of the dot product between two arrays: an array pointing in the radial direction and the 
    # one of the instantaneus velocity of the bicho
    angles = [ angle_function(unit_r, unit_v) for unit_r, unit_v  in zip(radial_direction_vector, velocity_direction_vector) ]    
        
    
    df_low.loc[df_low['particle'] == particle, 'angle'] =angles
    df_low.loc[df_low['particle'] == particle, 'vx'] =xders
    df_low.loc[df_low['particle'] == particle, 'vy'] =yders
    df_low.loc[df_low['particle'] == particle, 'v'] = np.sqrt(xders**2 + yders**2)

        
with open(path_for_saving + "/df_low.txt", "wb") as myFile:    #choose the name to save it 
    pickle.dump(df_low, myFile)
with open(path_for_saving +"/df_up.txt", "wb") as myFile:    #choose the name to save it 
    pickle.dump(df_up, myFile)
      

SyntaxError: invalid syntax (422650132.py, line 66)